In [1]:
import pandas as pd
from functions import *

In [2]:
# read in unique songs and Top100 daily charts of 8 countries
so = pd.read_csv('data/18k_complete.csv', header=0, index_col=False)
df = pd.read_csv('data/Top100_8_countries_forward.csv', header=0, index_col=False)

In [3]:
# show the top rows of the unique songs dataframe
so.head(3)

,title,artist,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity,album_name,album_release_date
0,Starboy,"The Weeknd, Daft Punk",5aAx2yezTd8zXrkmtKl66Z,0.681,0.594,7,-7.028,1,0.2820,0.165,0.000003,0.134,0.535,186.054,230453,28,Starboy,2016-11-25
1,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,5knuzwU65gJK7IF5yJsuaW,0.720,0.763,9,-4.068,0,0.0523,0.406,0.000000,0.180,0.742,101.965,251088,77,Rockabye (feat. Sean Paul & Anne-Marie),2016-10-21
2,Closer,"The Chainsmokers, Halsey",7BKLCZ1jbUBVqRi2FVlTVw,0.748,0.524,8,-5.599,1,0.0338,0.414,0.000000,0.111,0.661,95.010,244960,86,Closer,2016-07-29


In [4]:
# show the count of rows and columns 
so.shape

(18710, 18)

In [5]:
# select all songs with the audio features range of the cluster 3 (from machine learning 4 year backtest)
c7 = so.query('speechiness >= 0.0249 and speechiness <= 0.186 and valence >= 0.1 and valence <= 0.745 and danceability >= 0.284 and danceability <= 0.734 and energy >= 0.402 and energy <= 0.979').copy()
#3	Max. Danceability	0,734
#3	Min. Danceability	0,284
#3	Min. Energy	0,402
#3	Max. Energy	0,979
#3	Min. Speechiness	0,0249
#3	Max. Speechiness	0,186
#3	Min. Valence	0,1
#3	Max. Valence	0,745

In [6]:
# show the count of the rows and columns
c7.shape

(6662, 18)

In [7]:
# remove unneeded columns
c7.drop(['title', 'artist', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'popularity', 'album_name', 'album_release_date'], axis=1, inplace=True)

In [8]:
# show top rows
c7.head(3)

,track_id
1,5knuzwU65gJK7IF5yJsuaW
4,78rIJddV4X0HkNAInEcYde
5,5uCax9HTNlzGybIStD3vDh


In [9]:
# create 2 dataframes for the Top50 and Top100-51
jp = df.query('region == "Japan" and rank <=100 and rank >=51').copy()
jp1 = df.query('region == "Japan" and rank <=50').copy()

In [10]:
# show the count of the rows and columns
display(jp.shape, jp1.shape)

(16700, 8)

(16700, 8)

In [11]:
# merge to check fro matches
jp_c = jp.merge(c7, how='left', on='track_id', indicator=True) 
jp_c1 = jp1.merge(c7, how='left', on='track_id', indicator=True) 

In [12]:
# view top rows of dataframe Top100-51
jp_c.head()

,title,rank,date,artist,track_id,region,trend,streams,_merge
0,Stay Gold,68,2021-07-01,BTS,3Ys2PYl1wyPKQIwyqhP9cQ,Japan,MOVE_DOWN,54393.0,both
1,CITRUS,51,2021-07-01,Da-iCE,2IL2asrtx7XM1y8fu2RMnR,Japan,SAME_POSITION,77616.0,both
2,Alcohol-Free,52,2021-07-01,TWICE,3iPbmpKBMflYd7UjdYftoj,Japan,SAME_POSITION,72524.0,both
3,YOKAZE,53,2021-07-01,HENTAI SHINSHI CLUB,6tkSMR8bEIfEcJn7ybJtSP,Japan,SAME_POSITION,72500.0,both
4,Buddy,54,2021-07-01,HIRAIDAI,7xfPyBgb9om8iUooItxHl8,Japan,MOVE_UP,69382.0,left_only


In [13]:
# view top rows of dataframe Top50
jp_c1.head()

,title,rank,date,artist,track_id,region,trend,streams,_merge
0,Butter,1,2021-07-01,BTS,2bgTY4UwhfBYhGT4HUYStN,Japan,SAME_POSITION,333102.0,left_only
1,Dynamite,2,2021-07-01,BTS,4saklk6nie3yiGePpBwUoc,Japan,SAME_POSITION,247180.0,left_only
2,ドライフラワー,3,2021-07-01,優里,7dH0dpi751EoguDDg3xx6J,Japan,SAME_POSITION,246268.0,both
3,夜に駆ける,4,2021-07-01,YOASOBI,6MCjmGYlw6mQVWRFVgBRvB,Japan,SAME_POSITION,211785.0,left_only
4,怪物,5,2021-07-01,YOASOBI,06XQvnJb53SUYmlWIhUXUi,Japan,MOVE_UP,207869.0,left_only


In [15]:
# show the count of the rows and columns
display(jp_c.shape, jp_c1.shape)

(16700, 9)

(16700, 9)

In [16]:
# view count of merge results/matches Top100-51
jp_c.groupby('_merge').count()

,title,rank,date,artist,track_id,region,trend,streams
_merge,,,,,,,,
left_only,6386,6386,6386,6386,6386,6386,6386,6386
right_only,0,0,0,0,0,0,0,0
both,10314,10314,10314,10314,10314,10314,10314,10314


In [17]:
# view count of merge results/matches Top50
jp_c1.groupby('_merge').count()

,title,rank,date,artist,track_id,region,trend,streams
_merge,,,,,,,,
left_only,5687,5687,5687,5687,5687,5687,5687,5687
right_only,0,0,0,0,0,0,0,0
both,11013,11013,11013,11013,11013,11013,11013,11013


In [21]:
# save results in a dataframe Top100-51
jp_cb = jp_c.query('_merge == "both"')
jp_clo = jp_c.query('_merge == "left_only"')

In [22]:
# show the count of the rows and columns
display(jp_cb.shape, jp_clo.shape)

(10314, 9)

(6386, 9)

In [23]:
# save results in a dataframe Top50
jp_cb1 = jp_c1.query('_merge == "both"')
jp_clo1 = jp_c1.query('_merge == "left_only"')

In [24]:
# show the count of the rows and columns
display(jp_cb1.shape, jp_clo1.shape)

(11013, 9)

(5687, 9)